**Intro: Deep Learning on MNIST Data Set**

The problem: build a neural network that can recognize handwritten digits. The specific aim: Use the MNIST database of handwritten digits to train on the 60,000 total examples and test on a set of 10,000 examples (so 50,000 train and then 10,000 test). The data set can be found here: http://yann.lecun.com/exdb/mnist/

We are going to use the following to see what performs best (accuracy) on the MNIST data set:

* 1) Softmax Activation Function
* 2) Five Layer with Sigmoid
* 3) Five Layer with ReLu
* 4) and finally, Five Layer with ReLu with Dropout

To summarize the below:

* 1) We start with mnist 1.0 which used a single layer, a gradient descent optimizer, and a fixed learning rate of 0.005. This achieves about 92% accuracy on the test set.
* 2) To improve on this, we add 5 layers to the network. Each layer is initialized with a truncated_normal distribution for the weights and zeros for the biases. We still use a fixed learning rate here, but it is slightly smaller: 0.003. We also will change the optimizer to Adam instead of a vanilla Gradient descent optimizer. This improves the accuracy to around 98%. The activations for each layer are sigmoid activations.
* 3) In the next example, which is mnist 2.1, we have the same basic architecture with five layers. This time, however, the learning rate is not fixed. It exponentially decreases from 0.003 to 0.0001, over the course of all the 10001 training steps. We also initialize the biases differently, by using small positive values (0.1) instead of 0. The activation function for each layer here is ReLu instead of sigmoid. We notice we get a small accuracy improvement with ReLu.
* 4) Next is mnist 2.2, which introduces dropout. 2.2 is identical to 2.1, with the only exception that it has dropout. Overall the accuracy is up to something like 0.9817 at this point. But with dropout the gap between training and testing loss is not as wide.

**The First Iteration: Softmax (see file MNIST_1.0_Softmax.py)**

We start with a model that has only one layer and we have ten output neurons (a basic logistic regression). We use softmax as our activation function to compute the probability for ten neurons. We start with the most basic model that has only one layer, so it is essentially a regular logistic regression. Later on, we will add layers making it a neural network (with at least one 'hidden' layer, meaning one layer other than the input or output layers). All the logistic regression is doing here is taking linear regression, and then squashing the weights with a softmax activation function to have probabilities between 0 and 1. We get an accuracy around 92% - not great if you are relying on this for important text you need to have correct!

Each individual pixel is a feature in this model and the model figures out how much importance to give each pixel by assigning weights and biases. In the end this is squashed into a value that can be interpreted as a probability.A softmax function can’t be visualized on a graph because it is a function of more than two variables.

See full comments in the mnist_1.0_softmax.py code, but here is a SAMPLE OF THE CODE for this softmax one.

Here you are downloading the images and labels (supervised learning) into mnist test and train
**mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)**

placeholder for each input digit image (28x28 pixels)
**X = tf.placeholder(tf.float32, [None, 28, 28, 1])**

placeholder for the correct output digits 0-9
**Y_ = tf.placeholder(tf.float32, [None, 10])**

this is for storing each digit as a series of 784 bits
**W = tf.Variable(tf.zeros([784, 10]))**

bias is a constant we add to each neuron
**b = tf.Variable(tf.zeros([10]))**

This flattens the image into a single line of pixels
**XX = tf.reshape(X, [-1, 784])**

The model being a softmax activation function to get probabilities between 0 and 1: 
**Y = tf.nn.softmax(tf.matmul(XX, W) + b)**

<img src='1 Softmax.png' height= 60% width= 60%>

**The Second Iteration: Five Layer Sigmoid (see file MNIST_2.0_Five_Layers_Sigmoid.py)**

In the above example, we used softmax as the activation function because we had 10 classes to categorize. The main difference in this file was that we used multiple layers. In between each layer, as the activation function, we used sigmoid. In the last layer, because we are still classifying ten digits, we used softmax. One other difference is that in the first file above we used GradientDescent as the optimizer. In the second one, (and all future ones) we use the Adam optimizer. Adam introduces a couple of things that make gradient descent perform better, such as momentum. 

In the first example, we were getting an accuracy of around 92%. Here the accuracy has shot up to 98%. This is because in the first example (simple logistic regression) the model was not 'complex' enough to capture the essence of the data. Here we added five layers (using sigmoid on 4 of them and softmax on the last one), the number of parameters is much greater, so our model has more to work with. Sigmoid is the activation function traditionally used (until recently as we will see below with ReLu). The reason sigmoid is used is to introduce a 'nonlinearity' into the model. Also, the sigmoid function is continuous and differentiable. This makes it easier to compute gradients for backpropogation. But in the final layer, softmax is used instead because there are ten categories. Each one needs to be assigned a value between 0 and 1 representing the model's confidence in a particular outcome. 

So each neuron in the network has an input. This input is the sum of all the input values weighed by the weights, and with a bias term added to them. This sum is referred to as the logits. The activation function introduces an added complexity to the network. Suppose there were no activation functions at all - then the entire neural network would simply be a linear transformation. A composition of linear transformations is still just a linear transformation. And a linear transformation cannot capture all the kind of complexity we need for the problems that we’d like to solve. Adding an activation function essentially turns a neural network into a universal function - it can compute anything.

5 layers was chosen as a design choice - you could pick 3 or 10 and experiment to see what happens. Other hyperparamters you can tune are the learning rate and the number of neurons per layer. In reality nobody is quite sure yet how to choose the optimal values - it is an art at the moment.

However, in this example we still didn't make use of the image data properly because we flattened it into a vector (which is when CNNs come into play).

The sigmoid (shown below) is like a step function, it crosses the y axis when y=0.5 and has asymptotes at 0 and 1. It is a smoothed out version of a step function.

<img src='2 Five Layer Sigmoid.png' height= 60% width= 60%>

Sigmoid Activation Function

<img src='Sigmoid.png' height= 30% width= 30%>

**The Third Iteration: Five Layers with ReLu (see 2.1)**

Let's take a step back and think again about what we are doing in this project and then in this file with the new activiation function ReLu. Instead of using Sigmoid on the first 4 layers, we use ReLu which can be seen in the graph below as taking the value of 0 or the input, whichever is greater. ReLu performs better than Sigmoid and was initially discovered after inspiration for how the brain's neurons are either on or off (0 or 1). ReLu is pretty much always used instead of sigmoid today because it has empirically been observed to perform better as seen with the even higher accuracy (a touch better than 98%). A negative is if you look at the graph of training versus test loss, you will notice that the test loss does get worse and rises as you continue training your model (something we will fix with ReLu).

First, we get the data which is a bunch of images and its labels - this is typical for any supervised learning algorithm. 
* We create placeholders for the input data (X, and Y_,). 
* We create a network with 5 fully connected hidden layers.
* A hidden layer is any intermediate layer, i.e. one that is not an input or output layer.
* A fully connected layer is one which has each neuron in a layer connected to every neuron in the next layer.
* After the tensor operations for each layer (The tensors are the objects we are doing operations on, such as W1, B1, XX, etc; tensors are a general version of a matrix), we next have to define a loss function.
* The loss function could be something other than cross_entropy, but for classification problems it turns out that cross_entropy works really well.
* Once we have a loss function, we decide on an optimizer - in this case Adam.
* Then we can initialize the session and start the actual training.
* This is a very typical workflow in tensorflow - and it is mostly the same if you use something like Keras, but it’s a bit higher level.
* Next, we introduce the learning_rate decay. When training first begins, the learning_rate can be somewhat high, so that the loss function can be minimized more quickly. But as we get closer to the global minimum, we need to take smaller and smaller steps. Otherwise, we are at risk of bouncing around the edges of the valley because the steps are too large. You should expect to have better accuracy with a lower learning rate towards the end, in general.

<img src='2.1 Five Layers Relu.png' height= 60% width= 60%>

ReLu Activation Function

<img src='Relu.png' height= 30% width= 30%>

**The Fourth Iteration: Five Layers with ReLu with Dropout (see 2.2)**

Now, if you look at the graph of training versus test loss, you will notice that the test loss does NOT get worse as before as you continue training your model. The accuracy is high still but mostly the same as before. 

A look again at what we do in this file (outside of the code that deals with animation):

* We start off loading the data which is a collection of images and their labels as is typical for supervised learning.
* We create tensorflow placeholder objects to hold this data, but here we also have placeholders for learning rate (for learning rate decay) and pkeep( for dropout). These are parameters we are going to feed to the network once we run the session.
* Next, we have the initialization of weights and biases for each layer.
* The weights are initialized using a truncated normal distribution and the biases with small positive values.
* Next we set up the model, which involves performing the tensor operations. This time however, we introduce dropout which kills randomly chosen neurons in each iteration. This has the effect of thinning out the network and making it so that a particular neuron does not become too dependent on a pattern of firing off neighbouring neurons. So far the only regularization technique we have seen is dropout. There are other regularization techniques like early stopping, L1 or L2 regularization, and dropout. We only consider regularization techniques now because previously the architecture of the network wasn’t complex enough to overfit the data.
* The high level effect of this is to prevent the model from overfitting, and as you see above, it helps with reducing any difference between the test and train losses.
* Speaking of loss, next is where we define cross_entropy, our loss function.
* We also create variables for accuracy and the predictions (this is to evaluate our model).
* And finally the training step is created which links everything together before we initialize the variables and begin the session.
* In this case, the learning rate changes with each iteration, so we have it inside the training_step function inside the session. 

<img src='2.2 Five Layers Relu and Dropout.png' height= 60% width= 60%>

<img src='Dropout.png' height= 30% width= 30%>